In [1]:
import os
import sys
sys.path.append(os.path.abspath('../../../src/'))
from time import time

import pandas as pd
import numpy as np

import SortedNoDupeBedOverlap as bdo
import io_utils


In [2]:
def make_random_ocr():
    ocrs = []
    for i in range(1, 23):
        chrom_size = np.random.randint(1000, 10000)
        n_ocr = np.random.randint(100, 300)
        ints = np.sort(np.random.choice(chrom_size + 1, size = n_ocr, replace = False))
        ocrs += [('chr%i' % i, a, b) for a, b in zip(ints[::2], ints[1::2])]
    return ocrs

In [3]:
## bed0 missing chr 4

total_time = []

for seed in range(1000):
    np.random.seed(seed)
    chrom_size = 1000
    
    ## simulate random intervals
    bed0 = sorted(make_random_ocr())
    bed1 = sorted(make_random_ocr())
    bed0 = [p for p in bed0 if p[0] != 'chr4'] 
    
    ## compute pairwise overlap in a brute force way
    bed0_overlaps_bf = []
    for i, (c0, a0, b0) in enumerate(bed0):
        overlap = sum([bdo.overlap(a0, b0, a1, b1)  if c0 == c1 else 0 for (c1, a1, b1) in bed1])
        bed0_overlaps_bf.append(overlap / (b0 - a0))
    
    ## compute pairwise overlap with a linear algorithm
    tic = time()
    overlaps_linear = bdo.BedOverlap(bed0, bed1, 0.5)   
    total_time.append(time() - tic)
    
    assert set([p for p, o in zip(bed0, bed0_overlaps_bf) if o >= 0.5]) == set(overlaps_linear)
print('average run time: %.4f' % (sum(total_time) / len(total_time)) )

average run time: 0.0050
